In [1]:
import llm as L
L.TOKEN_COUNT_PATH = '/data/ai_club/team_3_2024-25/tokcounts2/'


In [2]:
# --- ONE MISTAKE --
# sentence = 'Haluan mennä kauppa.' # kauppa -> kauppaan
# sentence = 'Minä olen iloinen, koska sataa on' # extra "on"
sentence = 'Minä tykkään jäätelö' # jäätelö -> jäätelöstä
# sentence = 'Mä en osaa puhua suomi.' # suomi -> suomea
# sentence = 'Tämä on minun koirani' # correct
# sentence = "tämä minun koirani" # missing "on"
# sentence = "Han on minun opettajani" # Han -> Hän
# sentence = 'Tämä on minun koira' # koira -> koirani

# ctx = 'Mitä tämä on?'
# sentence = 'Tama on minun koirani' # Tama -> Tämä

# --- TWO MISTAKES ---

# sentence = 'Minä menee kouluun ja minä lukee kirja.' # menee -> menen & lukee -> luen
# sentence = "Tämä minun koira" # missing "on" & koira -> koirani
# sentence = 'Me menee kauppaan ja hän ostavat omenat.' # menee -> menemme & ostavat -> ostaa


In [10]:
def _extract_words(sentence):
    return sentence.split() # TODO : split better (handle punctuation)

def _highlight_word(idx, words):
    highlighted = words.copy()
    highlighted[idx] = '{{' + highlighted[idx] + '}}'
    highlighted = ' '.join(highlighted)
    return highlighted

def grade_per_word(sentence):
    llm = L.LLM('You are a Finnish language teacher.')

    cot = llm(
        'Is there a clear mistake in this sentence? Give (English) feedback for this sentence.\n\n' +
        # 'Give concise feedback/grading for this sentence:\n\n' +
        sentence
    )

    # print(cot)
    # print('\n-----')

    errs = llm(
        'Does your feedback indicate the presence of any improvements to be made?',
        response_format=['y/n']
    )['y/n']

    # print(errs)
    # print('\n-----')

    if errs == 'n':
        return [], [], None

    s = llm.save_state()

    words = _extract_words(sentence)
    incorrect = []
    for i in range(len(words)):
        sentence_w_highlight = _highlight_word(i, words)
        
        applies = llm(
            f'Does your feedback affect the highlighted word?\n\n{sentence_w_highlight}',
            response_format=['y/n']
        )['y/n']

        # print(applies, sentence_w_highlight)
        if applies == 'y':
            incorrect.append(i)

        llm.restore_state(s) # we shouldn't keep all words in history while grading them

    return words, incorrect, s

words, incorrect, state = grade_per_word(sentence)

In [24]:
def get_word_feedback(idx, words, state):
    llm = L.LLM()
    llm.restore_state(state)

    sentence_w_highlight = _highlight_word(idx, words)

    expl = llm(
        f'Give a consice explanation for why the highlighted word is incorrect\n\n{sentence_w_highlight}',
        response_format='stream'
    )

    return expl

expl = get_word_feedback(incorrect[0], words, state)
for t in expl: print(t, end='')

The word "jäätelö" is incorrect because it is in the nominative case, but the verb "tykkään" requires the elative case ("jäätelöstä") to express liking something.

---

In [140]:
llm = L.LLM('You are a yleiskieli Finnish language teacher.')
cot = llm(
    'Is there an obvious mistake in the following? Start by saying the correct version.\n\n' +
    sentence
)

print(cot)
print('\n-----')

corrected = llm(
    'Thanks for pointing that out. Please give me the correct version.',
    response_format=['corrected']
)['corrected']

print(corrected)
print('\n-----')

exp = llm(
    'Now plese give me your explanation without referring to the entire sentence. Do not respond in JSON',
    response_format='stream',
)

for t in exp:
    print(t, end='')

The correct version is:

Tämä on minun koira.

The mistake is in the word "Tama", which should be "Tämä". "Tämä" is a demonstrative pronoun meaning "this", whereas "Tama" is not a word in Finnish.

-----
Tämä on minun koira.

-----
The mistake is in the word "Tama", which should be "Tämä". "Tämä" is a demonstrative pronoun meaning "this", whereas "Tama" is not a word in Finnish.